<a href="https://colab.research.google.com/github/olga-terekhova/tdsb-calendar/blob/main/Integrate_Calendar_Template_PPTX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:

# ------------------------
# Parameters
# ------------------------


year = 2025
month = 10
workweek = 7 # try 5 for Mon–Fri only
week_start="Sunday" # or "Monday"


fontname = "Nunito Ultra-Bold"
fontpath = None # "Nunito-ExtraBold.ttf"

font_width_coef = 0.8

template_pptx = "October-2025-Template.pptx"
output_pptx = "calendar.pptx"

template_table_left_in = 0.32
template_table_top_in = 1.79
template_table_width_in = 10.36
template_table_height_in = 6.41

cell_left = 5
cell_top = 5

calendar_list = [
    {
        "name": "NoSchoolDays",
        "path": "calendar_no_school_days.ics",
        "style": {
            "bg_full_day": "#ffd1b3",   # orange
            "fg_full_day": "#000000",   # black
            "line_full_day": "#ffd1b3",   # orange
            "bg_timed": "#ADD8E6",      # light blue
            "fg_timed": "#000000",      # black
            "line_timed": "#000000",      # black
            "fontsize": 14
        }
    },
    {
        "name": "SchoolDays",
        "path": "calendar_school_days.ics",
        "style": {
            "bg_full_day":  "#9fdfbf",   # light green
            "fg_full_day": "#000000",   # black
            "line_full_day": "#9fdfbf",   # light green
            "bg_timed": "#FFB6C1",      # light pink
            "fg_timed": "#000000",      # black
            "line_timed": "#000000",      # black
            "fontsize": 14
            }
    },
    {
        "name": "Daily",
        "path": "calendar_schedule.ics",
        "style": {
            "bg_full_day": "#FFFFFF",      # white
            "fg_full_day": "#000000",   # black
            "line_full_day": "#9fdfbf",   # light green
            "bg_timed": "#FFFFFF",      # white
            "fg_timed": "#000000",      # black
            "line_timed": "#9fdfbf",   # light green
            "fontsize": 14
            }
    }

]



In [122]:
!pip install icalendar pytz

In [123]:
!pip install python-pptx

In [124]:
!pip install fonttools

In [125]:
from datetime import datetime
from pathlib import Path
from icalendar import Calendar
import calendar
import pytz

import os
from collections import defaultdict

from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.shapes import MSO_SHAPE
from pptx.enum.text import PP_ALIGN
from pptx.oxml import parse_xml
from pptx.oxml.ns import qn
from pptx.oxml.ns import nsdecls

from fontTools.ttLib import TTFont

In [126]:
def parse_ics_file(calendar, priority, year, month):
    """Parse events from one ICS file filtered by month/year."""

    events = []
    with open(calendar["path"], "rb") as f:
        cal = Calendar.from_ical(f.read())
        for component in cal.walk():
            if component.name == "VEVENT":
                start = component.get("DTSTART").dt
                summary = str(component.get("SUMMARY"))
                all_day = not isinstance(start, datetime)

                if isinstance(start, datetime):
                    start = start.astimezone(pytz.UTC) if start.tzinfo else start

                if start.year == year and start.month == month:
                    event_style = {
                        "fontsize": calendar["style"]["fontsize"],
                        "bg": calendar["style"]["bg_full_day"] if all_day else calendar["style"]["bg_timed"],
                        "fg": calendar["style"]["fg_full_day"] if all_day else calendar["style"]["fg_timed"],
                        "line": calendar["style"]["line_full_day"] if all_day else calendar["style"]["line_timed"]
                    }
                    events.append({
                        "calendar_name": calendar["name"],
                        "calendar_order": priority,
                        "day": start.day,
                        "event_name": summary,
                        "is_full_day": all_day,
                        "time": None if all_day else start.strftime("%H:%M"),
                        "weekday": start.strftime("%A"),
                        "style": event_style
                    })
    return events


def collect_events(calendar_list, year, month):
    """Union events from all calendars with sorting."""
    all_events = []
    for priority, calendar in enumerate(calendar_list):
        all_events.extend(parse_ics_file(calendar, priority, year, month))

    all_events.sort(
        key=lambda e: (
            e["day"],
            0 if e["is_full_day"] else 1,
            e["time"] if e["time"] else "99:99",
            e["calendar_order"]
        )
    )
    return all_events



In [127]:
def calendar_layout(year, month, week_start="Monday", workweek=7):
    """
    Calculate the calendar grid layout for a monthly calendar.

    Parameters
    ----------
    year : int
        Year of the calendar.
    month : int
        Month of the calendar (1–12).
    week_start : str
        "Monday" or "Sunday" — only used when workweek=7.
        If workweek=5, calendar always starts on Monday.
    workweek : int
        5 or 7 — number of days shown per row.

    Returns
    -------
    layout : dict
        {
          "rows": int,
          "cols": int,
          "mapping": {day: (row, col), ...}
        }
    """
    if workweek not in (5, 7):
        raise ValueError("workweek must be 5 or 7")

    # Python calendar: Monday=0 ... Sunday=6
    first_weekday, num_days = calendar.monthrange(year, month)

    mapping = {}
    row = 0

    if workweek == 7:
        # Adjust start-of-week
        start_idx = 0 if week_start == "Monday" else 6

        for day in range(1, num_days + 1):
            weekday = (first_weekday + (day - 1)) % 7
            col = (weekday - start_idx) % 7
            mapping[day] = (row, col)

            if col == 6:
                row += 1

        cols = 7

    else:  # workweek == 5 (always Monday–Friday)
        for day in range(1, num_days + 1):
            weekday = (first_weekday + (day - 1)) % 7
            if weekday in (5, 6):  # Skip Saturday, Sunday
                continue
            col = weekday  # Monday=0 ... Friday=4
            mapping[day] = (row, col)

            if col == 4:
                row += 1

        cols = 5

    rows = max(r for r, _ in mapping.values()) + 1 if mapping else 0

    return {"rows": rows, "cols": cols, "mapping": mapping}




In [128]:
def hex_to_rgb(hex_color):
    """Convert hex string #RRGGBB to tuple of floats (r, g, b) in 0–1."""
    if not isinstance(hex_color, str):
        return None
    hex_color = hex_color.lstrip("#")
    if len(hex_color) != 6:
        return None
    try:
        r = int(hex_color[0:2], 16) / 255
        g = int(hex_color[2:4], 16) / 255
        b = int(hex_color[4:6], 16) / 255
        return (r, g, b)
    except ValueError:
        return None

In [129]:
def get_font(font_path):
    """
    Reads global variable `event_font` (path to TTF file).
    Returns a dict with keys:
      - "custom"   : bool, True if using custom TTF
      - "fontname" : str, the system-recognized font family name
      - "font_obj" : TTFont object
    """

    # global event_font

    if font_path and os.path.isfile(font_path):
        # Parse the TTF file
        font_obj = TTFont(font_path)

        # Extract full font name (NameID 4 = Full font name)
        name_records = font_obj['name'].names
        fontname = None
        for record in name_records:
            if record.nameID == 4:  # Full font name
                try:
                    fontname = record.string.decode(record.getEncoding())
                except UnicodeDecodeError:
                    fontname = record.string.decode('utf-8', errors='ignore')
                break
        print(fontname)
        if fontname is None:
            fontname = os.path.basename(font_path).rsplit('.', 1)[0]
        print(fontname)
        return {
            "custom": True,
            "fontname": fontname,
            "font_obj": font_obj
        }
    else:
        # Fallback
        return {
            "custom": False,
            "fontname": "Calibri",
            "font_obj": None
        }

In [130]:
res = get_font("Nunito-ExtraBold.ttf")
print(res)

Nunito ExtraBold
Nunito ExtraBold
{'custom': True, 'fontname': 'Nunito ExtraBold', 'font_obj': <fontTools.ttLib.ttFont.TTFont object at 0x7853e4a91970>}


In [131]:
from PIL import ImageFont

def measure_text_width(font_path, text, fontsize):
    """
    Measure pixel width of `text` rendered in a TTF font using PIL.

    Parameters
    ----------
    font_path : str
        Path to the .ttf font file.
    text : str
        The text to measure.
    fontsize : int
        Font size in points.

    Returns
    -------
    int
        Text width in pixels.
    """
    font = ImageFont.truetype(font_path, fontsize)  # first param = path to .ttf
    bbox = font.getbbox(text)  # returns (x0, y0, x1, y1)
    return bbox[2] - bbox[0]   # width


In [132]:
res_m = measure_text_width("Nunito-ExtraBold.ttf", "Thanksgiving", 14)
print(res_m)

90


In [133]:
# Conversion helpers: px-like → Inches
def px_to_inches(px):
        return px / 96.0   # assume 96 DPI

# Conversion helpers: Inches → px-like
def inches_to_px(inches):
        return inches * 96.0   # assume 96 DPI

In [134]:
def set_corner_radius(shape, corner_radius_val=10000):
    """
    Adjust corner radius for a rounded rectangle shape.
    corner_radius_val: int, 0..50000
    """
    # DrawingML namespace
    NS = "http://schemas.openxmlformats.org/drawingml/2006/main"

    spPr = shape.element.spPr
    prstGeom = spPr.find(f"{{{NS}}}prstGeom")
    if prstGeom is None:
        raise ValueError("Shape has no prstGeom element")

    avLst = prstGeom.find(f"{{{NS}}}avLst")
    if avLst is None:
        raise ValueError("Shape has no avLst element")

    # Clear existing adjustments
    for adj in list(avLst):
        avLst.remove(adj)

    # Add new adjustment with proper namespace
    adj_xml = (
        f'<a:gd xmlns:a="{NS}" name="adj" fmla="val {corner_radius_val}"/>'
    )
    avLst.append(parse_xml(adj_xml))

In [135]:
def create_calendar_pptx(output_pptx, year, month, layout, events, input_pptx=None):
    """
    Create a PPTX calendar with events placed in grid cells.

    Parameters
    ----------
    output_pptx : str
        Output PPTX file path.
    year : int
        Year of the calendar.
    month : int
        Month of the calendar.
    layout : dict
        From calendar_layout(): {"rows", "cols", "mapping"}.
    events : list of dict
        Events with fields: "day", "event_name", etc.
    input_pptx : str or None
        Path to an existing PPTX. If provided, first slide is used as background.
    """

    # Extract grid dimensions
    rows, cols = layout["rows"], layout["cols"]
    mapping = layout["mapping"]

    # Open template or new presentation
    if input_pptx:
        prs = Presentation(input_pptx)
    else:
        prs = Presentation()

    slide = prs.slides[0]  # work on first slide

    # Calculate layout
    table_width = inches_to_px(template_table_width_in)
    table_height = inches_to_px(template_table_height_in)
    table_left = inches_to_px(template_table_left_in)
    table_top = inches_to_px(template_table_top_in)

    cell_w = table_width / cols
    cell_h = table_height / rows

    # Group events by day
    day_events = defaultdict(list)
    for e in events:
        day_events[e["day"]].append(e)

    # Add events to slide
    for day, (r, c) in mapping.items():
        x0 = (c * cell_w + table_left + cell_left)
        y0 = (r * cell_h + table_top + cell_top)

        for e in day_events.get(day, []):
            label = e["event_name"]
            fontsize = e["style"]["fontsize"]

            bg_color = e["style"]["bg"].lstrip("#")
            fg_color = e["style"]["fg"].lstrip("#")
            line_color = e["style"]["line"].lstrip("#")

            # Estimate text box width
            if fontpath:
              text_width = measure_text_width(fontpath, label, fontsize)
            else:
              text_width = len(label) * fontsize * font_width_coef
            padding_w = fontsize
            padding_h = fontsize

            left = Inches(px_to_inches(x0))
            top = Inches(px_to_inches(y0))
            width = Inches(px_to_inches(text_width + padding_w))
            height = Inches(px_to_inches(fontsize + padding_h))

            # Add rectangle with rounded corners
            shape = slide.shapes.add_shape(
                MSO_SHAPE.ROUNDED_RECTANGLE,
                left, top, width, height
            )
            shape.shadow.inherit = False
            set_corner_radius(shape, corner_radius_val=40000)

            # Fill color
            fill = shape.fill
            fill.solid()
            fill.fore_color.rgb = RGBColor.from_string(bg_color)

            # Border color
            line = shape.line
            line.color.rgb = RGBColor.from_string(line_color)

            # Add text
            text_frame = shape.text_frame
            text_frame.clear()  # ensure it's empty
            text_frame.margin_left = Inches(px_to_inches(5))
            text_frame.margin_right  = Inches(px_to_inches(5))
            p = text_frame.paragraphs[0]
            p.alignment = PP_ALIGN.LEFT
            run = p.add_run()
            run.text = label

            font = run.font
            font.name = fontname
            font.size = Pt(fontsize)
            font.color.rgb = RGBColor.from_string(fg_color)

            # vertical spacing
            y0 += fontsize + fontsize

    prs.save(output_pptx)
    print(f"PPTX saved: {output_pptx}")



In [136]:
# Run event collection and integration into the template
events = collect_events(calendar_list, year, month)

layout = calendar_layout(year, month, week_start=week_start, workweek=workweek)

create_calendar_pptx(output_pptx, year, month, layout, events, template_pptx)


PPTX saved: calendar.pptx
